In [38]:
import numpy as np
import plotly.graph_objects as go
from tqdm import tqdm
import polars as pl

In [39]:
actif = 'GOOGL'
limite = 0


In [40]:
df = pl.read_parquet(f'/home/janis/3A/EA/HFT_QR_RL/data/smash3/data/csv/NASDAQ/{actif}/20240927.parquet')

In [41]:
df = df.filter(
    (pl.col('symbol') == actif) &
    (pl.col('depth') == limite) &
    (pl.col('side').is_in(['A','B']))
)

# No need to parse ts_event since it's already datetime[ns, UTC]
df = df.filter(
    (pl.col('ts_event').dt.hour() >= 14) & 
    (pl.col('ts_event').dt.hour() < 19)
)

In [18]:
# import pandas as pd

# # Assurez-vous que le DataFrame est trié par 'ts_event'
# df_sorted = df_.sort_values(by='ts_event').reset_index(drop=True)

# # Spécifiez le timestamp cible
# target_timestamp = pd.Timestamp('2024-09-27 14:00:00.707592949+00:00')

# # Localiser l'index de la ligne qui correspond au timestamp cible
# target_index = df_sorted[df_sorted['ts_event'] == target_timestamp].index

# # Initialiser une liste pour stocker les événements avant et après
# events_around_target = []

# # Si le timestamp cible existe dans le DataFrame, récupérez les événements avant et après
# if len(target_index) > 0:
#     target_index = target_index[0]  # Prendre le premier match si plusieurs existent

#     # Événement juste avant
#     if target_index > 0:
#         event_before = df_sorted.iloc[target_index - 1]
#         events_around_target.append(event_before)

#     # Événement juste après
#     if target_index < len(df_sorted) - 1:
#         event_after = df_sorted.iloc[target_index + 1]
#         events_around_target.append(event_after)

#     # Convertir les événements en DataFrame
#     df_events_around_target = pd.DataFrame(events_around_target)
#     print("Événements juste avant et juste après le timestamp cible:")
# else:
#     print("Le timestamp cible n'est pas présent dans le DataFrame.")


In [19]:
# import pandas as pd

# # Assurez-vous que le DataFrame est trié par 'ts_event'
# df_sorted = df.sort_values(by='ts_event').reset_index(drop=True)

# # Spécifiez le timestamp cible
# target_timestamp = pd.Timestamp('2024-09-27 14:00:00.707592949+00:00')

# # Filtrer pour obtenir l'événement immédiatement après le timestamp cible
# event_after = df_sorted[df_sorted['ts_event'] > target_timestamp].head(1)

# event_after.head()


In [20]:
# df_events_around_target.head()

In [21]:
# df[df['ts_event'] == pd.Timestamp('2024-09-27 14:00:00.707592949+00:00')].head(10)

In [22]:
def visu_1():
    # Get numpy arrays for plotting
    size_bid = df['bid_sz_00'].to_numpy()
    size_ask = df['ask_sz_00'].to_numpy()
    time = df['ts_event'].cast(pl.Datetime).to_numpy()

    # First plot - Queue sizes
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=time, y=size_bid, mode='lines', name='Bid', showlegend=True))
    fig.add_trace(go.Scatter(x=time, y=size_ask, mode='lines', name='Ask', showlegend=True))
    fig.update_layout(title='Taille de queues', xaxis_title='size', yaxis_title='intensity', showlegend=True)
    fig.show()

    # Get data for second plot
    trades_df = df.filter(pl.col('action') == 'T')
    price_1 = trades_df['price'].to_numpy() 
    time_price_1 = trades_df['ts_event'].cast(pl.Datetime).to_numpy()
    
    # Get price levels
    bid_ask_cols = ['bid_px_00', 'ask_px_00', 'bid_px_01', 'ask_px_01', 
                    'bid_px_02', 'ask_px_02', 'bid_px_03', 'ask_px_03']
    price_arrays = {col: df[col].to_numpy() for col in bid_ask_cols}

    # Second plot - Prices
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=time_price_1, y=price_1, mode='lines', name='Prix', showlegend=True))
    
    for i, (name, values) in enumerate(price_arrays.items()):
        side = 'Bid' if 'bid' in name else 'Ask'
        level = name[-2:]
        fig.add_trace(go.Scatter(x=time, y=values, mode='lines', 
                               name=f'{side} {level}', showlegend=True))
    
    fig.update_layout(title='Bid-ask', xaxis_title='size', yaxis_title='intensity', showlegend=True)
    fig.show()


In [23]:
visu_1()

KeyboardInterrupt: 

In [42]:
df_ = df.select([
    'ts_event', 'action', 'side', 'size', 'price',
    f'bid_px_0{limite}', f'ask_px_0{limite}', 
    f'bid_sz_0{limite}', f'ask_sz_0{limite}',
    f'bid_ct_0{limite}', f'ask_ct_0{limite}',
    f'bid_px_0{limite+1}', f'ask_px_0{limite+1}', 
    f'bid_sz_0{limite+1}', f'ask_sz_0{limite+1}',
    f'bid_ct_0{limite+1}', f'ask_ct_0{limite+1}'
])

In [43]:

df_ = df_.filter(pl.col('bid_sz_00') > 1000)
df_ = df_.filter(pl.col('action') == 'T')
df_.head(100)

ts_event,action,side,size,price,bid_px_00,ask_px_00,bid_sz_00,ask_sz_00,bid_ct_00,ask_ct_00,bid_px_01,ask_px_01,bid_sz_01,ask_sz_01,bid_ct_01,ask_ct_01
"datetime[ns, UTC]",str,str,u32,f64,f64,f64,u32,u32,u32,u32,f64,f64,u32,u32,u32,u32
2024-09-27 14:02:36.262910706 UTC,"""T""","""B""",97,164.21,164.2,164.21,1046,97,11,1,164.19,164.22,620,203,5,4
2024-09-27 14:03:02.555153387 UTC,"""T""","""B""",50,164.32,164.31,164.32,1155,50,1,1,164.3,164.33,30,111,2,3
2024-09-27 14:03:02.756882639 UTC,"""T""","""B""",50,164.32,164.31,164.32,1155,100,1,1,164.3,164.33,152,131,4,3
2024-09-27 14:03:08.077629988 UTC,"""T""","""B""",11,164.33,164.31,164.33,1438,18,7,2,164.3,164.34,742,21,6,2
2024-09-27 14:03:08.077629988 UTC,"""T""","""B""",7,164.33,164.31,164.33,1438,7,7,1,164.3,164.34,742,21,6,2
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2024-09-27 15:09:30.793000403 UTC,"""T""","""A""",100,165.0,165.0,165.01,1567,120,47,2,164.99,165.02,180,305,4,6
2024-09-27 15:09:30.793000403 UTC,"""T""","""A""",10,165.0,165.0,165.01,1467,120,46,2,164.99,165.02,180,305,4,6
2024-09-27 15:09:30.793000403 UTC,"""T""","""A""",1,165.0,165.0,165.01,1457,120,45,2,164.99,165.02,180,305,4,6


In [51]:
df_ = df_.with_columns([
    pl.col('ts_event').diff().dt.total_seconds().alias('time_diff')
])
df_.head(100)

ts_event,action,side,size,price,bid_px_00,ask_px_00,bid_sz_00,ask_sz_00,bid_ct_00,ask_ct_00,bid_px_01,ask_px_01,bid_sz_01,ask_sz_01,bid_ct_01,ask_ct_01,time_diff,price_same,price_opposite,size_same,size_opposite,nb_ppl_same,nb_ppl_opposite,imbalance,indice,bid_sz_00_diff,ask_sz_00_diff,diff_price,Mean_price_diff
"datetime[ns, UTC]",str,str,u32,f64,f64,f64,u32,u32,u32,u32,f64,f64,u32,u32,u32,u32,i64,f64,f64,u32,u32,u32,u32,f64,i64,i64,i64,f64,f64
2024-09-27 14:02:36.262910706 UTC,"""T""","""B""",97,164.21,164.2,164.21,1046,97,11,1,164.19,164.22,620,203,5,4,null,164.2,164.21,1046,97,11,1,3.7576e6,0,null,null,null,null
2024-09-27 14:03:02.555153387 UTC,"""T""","""B""",50,164.32,164.31,164.32,1155,50,1,1,164.3,164.33,30,111,2,3,26,164.31,164.32,1155,50,1,1,3.5643e6,1,109,-47,0.11,null
2024-09-27 14:03:02.756882639 UTC,"""T""","""B""",50,164.32,164.31,164.32,1155,100,1,1,164.3,164.33,152,131,4,3,0,164.31,164.32,1155,100,1,1,3.4223e6,2,0,50,0.0,null
2024-09-27 14:03:08.077629988 UTC,"""T""","""B""",11,164.33,164.31,164.33,1438,18,7,2,164.3,164.34,742,21,6,2,5,164.31,164.33,1438,18,7,2,2.9498e6,3,283,-82,0.01,null
2024-09-27 14:03:08.077629988 UTC,"""T""","""B""",7,164.33,164.31,164.33,1438,7,7,1,164.3,164.34,742,21,6,2,0,164.31,164.33,1438,7,7,1,2.9723e6,4,0,-11,0.0,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2024-09-27 15:09:30.793000403 UTC,"""T""","""A""",100,165.0,165.0,165.01,1567,120,47,2,164.99,165.02,180,305,4,6,0,165.01,165.0,120,1567,2,47,2.5459e6,95,-1,0,0.0,-0.021
2024-09-27 15:09:30.793000403 UTC,"""T""","""A""",10,165.0,165.0,165.01,1467,120,46,2,164.99,165.02,180,305,4,6,0,165.01,165.0,120,1467,2,46,2.7063e6,96,-100,0,0.0,-0.021
2024-09-27 15:09:30.793000403 UTC,"""T""","""A""",1,165.0,165.0,165.01,1457,120,45,2,164.99,165.02,180,305,4,6,0,165.01,165.0,120,1457,2,45,2.7235e6,97,-10,0,0.0,-0.021


In [52]:
# Check if required columns exist
required_columns = [
    'side', f'ask_px_0{limite}', f'bid_px_0{limite}', 
    f'ask_sz_0{limite}', f'bid_sz_0{limite}',
    f'ask_ct_0{limite}', f'bid_ct_0{limite}',
    'price', 'action', 'size'
]

for col in required_columns:
    if col not in df_.columns:
        print(f"Missing required column: {col}")

# First add all columns except those depending on diff_price
df_ = df_.with_columns([
    pl.when(pl.col('side') == 'A')
      .then(pl.col(f'ask_px_0{limite}'))
      .otherwise(pl.col(f'bid_px_0{limite}'))
      .alias('price_same'),
    pl.when(pl.col('side') == 'A')
      .then(pl.col(f'bid_px_0{limite}'))
      .otherwise(pl.col(f'ask_px_0{limite}'))
      .alias('price_opposite'),
    pl.when(pl.col('side') == 'A')
      .then(pl.col(f'ask_sz_0{limite}'))
      .otherwise(pl.col(f'bid_sz_0{limite}'))
      .alias('size_same'),
    pl.when(pl.col('side') == 'A')
      .then(pl.col(f'bid_sz_0{limite}'))
      .otherwise(pl.col(f'ask_sz_0{limite}'))
      .alias('size_opposite'),
    pl.when(pl.col('side') == 'A')
      .then(pl.col(f'ask_ct_0{limite}'))
      .otherwise(pl.col(f'bid_ct_0{limite}'))
      .alias('nb_ppl_same'),
    pl.when(pl.col('side') == 'A')
      .then(pl.col(f'bid_ct_0{limite}'))
      .otherwise(pl.col(f'ask_ct_0{limite}'))
      .alias('nb_ppl_opposite'),
    ((pl.col(f'ask_sz_0{limite}') - pl.col(f'bid_sz_0{limite}')) / 
     (pl.col(f'ask_sz_0{limite}') + pl.col(f'bid_sz_0{limite}'))).alias('imbalance'),
    pl.arange(0, pl.len()).alias('indice'),
    pl.col(f'bid_sz_0{limite}').diff().alias(f'bid_sz_0{limite}_diff'),
    pl.col(f'ask_sz_0{limite}').diff().alias(f'ask_sz_0{limite}_diff')
])

# Check if price column exists before adding diff_price
if 'price' not in df_.columns:
    print("Missing price column for diff_price calculation")
else:
    df_ = df_.with_columns([
        pl.col('price').diff().alias('diff_price')
    ])

# Check if diff_price exists before calculating Mean_price_diff  
if 'diff_price' not in df_.columns:
    print("Missing diff_price column for Mean_price_diff calculation")
else:
    df_ = df_.with_columns([
        pl.col('diff_price').rolling_mean(10).shift(1).alias('Mean_price_diff')
    ])

# Check columns needed for status calculation
status_columns = ['action', 'side', f'bid_sz_0{limite}_diff', f'ask_sz_0{limite}_diff', 'size']
if all(col in df_.columns for col in status_columns):
    df_ = df_.with_columns([
        pl.when(
            ((pl.col('action') == 'T') & 
             ((pl.col('side') == 'B') & (pl.col(f'bid_sz_0{limite}_diff') == -pl.col('size')) |
              (pl.col('side') == 'A') & (pl.col(f'ask_sz_0{limite}_diff') == -pl.col('size')))) |
            ((pl.col('action') == 'A') &
             ((pl.col('side') == 'B') & (pl.col(f'bid_sz_0{limite}_diff') == pl.col('size')) |
              (pl.col('side') == 'A') & (pl.col(f'ask_sz_0{limite}_diff') == pl.col('size')))) |
            ((pl.col('action') == 'C') &
             ((pl.col('side') == 'B') & (pl.col(f'bid_sz_0{limite}_diff') == -pl.col('size')) |
              (pl.col('side') == 'A') & (pl.col(f'ask_sz_0{limite}_diff') == -pl.col('size'))))
        ).then('OK').otherwise('NOK').alias('status')
    ])
else:
    print("Missing columns needed for status calculation")

# Check columns needed for new_limite and price_middle
if all(col in df_.columns for col in [f'bid_px_0{limite}', f'ask_px_0{limite}']):
    df_ = df_.with_columns([
        pl.when(
            (pl.col(f'bid_px_0{limite}').diff() > 0) | 
            (pl.col(f'ask_px_0{limite}').diff() > 0)
        ).then('new_limite').otherwise('n').alias('new_limite'),
        ((pl.col(f'ask_px_0{limite}') - pl.col(f'bid_px_0{limite}'))/2).alias('price_middle')
    ])
else:
    print("Missing columns needed for new_limite and price_middle calculations")

df_.head(1)


ColumnNotFoundError: OK

Resolved plan until failure:

	---> FAILED HERE RESOLVING 'with_columns' <---
DF ["ts_event", "action", "side", "size"]; PROJECT */30 COLUMNS; SELECTION: None

In [12]:
import pandas as pd

df_ = df_.reset_index(drop=True)

new_rows = []
lims = []
i = 0
total_rows = len(df_)
while i < total_rows-1:

    if i % (total_rows//10) == 0:
        print(f"Progression du code de merde: {int((i/total_rows)*100)}%")
    current_timestamp = df_.loc[i, 'ts_event']
    indices_group = [i]
    j = i+1
    
    while j < total_rows and df_.loc[j, 'ts_event'] == current_timestamp:
        indices_group.append(j)
        j += 1

    if len(indices_group) > 1:
        events_group = df_.iloc[indices_group]
        all_trades_then_cancel = all(events_group['action'].iloc[:-1] == 'T') and events_group['action'].iloc[-1] == 'C'
        all_trades = all(events_group['action'] == 'T')
        complex_trades_cancels = (
            events_group['action'].iloc[-1] == 'C' and
            all(
                events_group['action'].iloc[start:k].eq('T').all()
                for k, action in enumerate(events_group['action'])
                if action == 'C' and (start := events_group['action'].iloc[:k].last_valid_index()) is not None
            )
        )
        no_new_limite = 'new_limite' not in events_group['new_limite'].values
        
        complex_trades_cancels_not_ended_by_cancel = (
            all(
                events_group['action'].iloc[start:k].eq('T').all()
                for k, action in enumerate(events_group['action'])
                if action == 'C' and (start := events_group['action'].iloc[:k].last_valid_index()) is not None
            )
        )

        if all_trades_then_cancel and complex_trades_cancels and no_new_limite:
            total_size = events_group.loc[events_group['action'] == 'T', 'size'].sum()
            new_row = events_group.iloc[-1].copy()
            new_row['size'] = total_size
            new_row['action'] = 'T'
            new_rows.append(new_row)
            lims.extend((np.unique(events_group['new_limite'].to_numpy())))

        elif complex_trades_cancels and no_new_limite:
            total_size = events_group.loc[events_group['action'] == 'T', 'size'].sum()
            new_row = events_group.iloc[-1].copy()
            new_row['size'] = total_size
            new_row['action'] = 'T'
            new_rows.append(new_row)
            lims.extend((np.unique(events_group['new_limite'].to_numpy())))

        elif all_trades_then_cancel and not no_new_limite:
            limite_ = events_group['new_limite'].values
            bonne_limite = [0]+[l for l in range(len(limite_)) if limite_[l] == 'new_limite']
            for k in range(len(bonne_limite) - 1):
                start_index = bonne_limite[k]
                end_index = bonne_limite[k + 1]
                total_size = events_group[start_index:end_index].query("action == 'T'")['size'].sum()
                new_row = events_group.iloc[-1].copy()
                new_row['size'] = total_size
                new_row['action'] = 'T'
                new_row['new_limite'] = 'limite_épuisée'
                if new_row['side'] == 'B':
                    new_row[f'bid_sz_0{limite}'] = 0
                elif new_row['side'] == 'A':
                    new_row[f'ask_sz_0{limite}'] = 0
                new_rows.append(new_row)
            total_size = events_group.loc[bonne_limite[-1]:].query("action == 'T'")['size'].sum()
            new_row = events_group.iloc[-1].copy()
            new_row['size'] = total_size
            new_row['action'] = 'T'
            new_row['new_limite'] = 'new_limite'
            new_rows.append(new_row)

        elif complex_trades_cancels and not no_new_limite:
            limite_ = events_group['new_limite'].values
            bonne_limite = [0]+[l for l in range(len(limite_)) if limite_[l] == 'new_limite']
            for k in range(len(bonne_limite) - 1):
                start_index = bonne_limite[k]
                end_index = bonne_limite[k + 1]
                total_size = events_group[start_index:end_index].query("action == 'T'")['size'].sum()
                new_row = events_group.iloc[-1].copy()
                new_row['size'] = total_size
                new_row['action'] = 'T'
                new_row['new_limite'] = 'limite_épuisée'
                if new_row['side'] == 'B':
                    new_row[f'bid_sz_0{limite}'] = 0
                elif new_row['side'] == 'A':
                    new_row[f'ask_sz_0{limite}'] = 0
                new_rows.append(new_row)
            total_size = events_group.loc[bonne_limite[-1]:].query("action == 'T'")['size'].sum()
            new_row = events_group.iloc[-1].copy()
            new_row['size'] = total_size
            new_row['action'] = 'T'
            new_row['new_limite'] = 'new_limite'
            new_rows.append(new_row)

        elif all_trades:
            if df_.iloc[j]['new_limite'] == 'new_limite':
                total_size = events_group.loc[events_group['action'] == 'T', 'size'].sum()
                new_row = events_group.iloc[-1].copy()
                new_row['size'] = 0
                new_row['action'] = 'T'
                new_rows.append(new_row)
            else:
                total_size = events_group.loc[events_group['action'] == 'T', 'size'].sum()
                new_row = events_group.iloc[-1].copy()
                new_row['size'] = total_size
                new_row['action'] = 'T'
                new_rows.append(new_row)

        elif complex_trades_cancels_not_ended_by_cancel:
            if df_.iloc[j]['new_limite'] == 'new_limite':
                total_size = events_group.loc[events_group['action'] == 'T', 'size'].sum()
                new_row = events_group.iloc[-1].copy()
                new_row['size'] = 0
                new_row['action'] = 'T'
                new_rows.append(new_row)
            else:
                total_size = events_group.loc[events_group['action'] == 'T', 'size'].sum()
                new_row = events_group.iloc[-1].copy()
                new_row['size'] = total_size
                new_row['action'] = 'T'
                new_rows.append(new_row)

        else:
            new_rows.extend(events_group.to_dict(orient='records'))

        i = j
    else:
        new_rows.append(df_.iloc[i].to_dict())
        i += 1

print("Traitement terminé à 100% MGL")
# lims = np.array(lims).flatten()
# print(len(lims))
# print(lims)

Progression du code de merde: 0%
Progression du code de merde: 9%
Progression du code de merde: 19%
Progression du code de merde: 39%
Progression du code de merde: 49%
Progression du code de merde: 59%
Progression du code de merde: 69%
Progression du code de merde: 79%
Progression du code de merde: 89%
Traitement terminé à 100% MGL


In [13]:
standardized_series_list = []
for item in tqdm(new_rows):
    if isinstance(item, dict):
        standardized_series_list.append(pd.Series(item))
    elif isinstance(item, pd.Series):
        standardized_series_list.append(item)
    else:
        raise ValueError("L'élément de la liste n'est ni un dictionnaire ni une série.")

df__ = pd.concat(standardized_series_list, axis=1).T.reset_index(drop=True)


100%|██████████| 583543/583543 [00:25<00:00, 22899.91it/s]


In [14]:
df__.head(2)

,ts_event,action,side,size,price,bid_px_00,ask_px_00,bid_sz_00,ask_sz_00,bid_ct_00,ask_ct_00,bid_px_01,ask_px_01,bid_sz_01,ask_sz_01,bid_ct_01,ask_ct_01,time_diff,price_same,price_opposite,size_same,size_opposite,nb_ppl_same,nb_ppl_opposite,diff_price,Mean_price_diff,imbalance,indice,bid_sz_00_diff,ask_sz_00_diff,status,new_limite
0,2024-09-27 14:00:00.001848092+00:00,T,A,45,163.98,163.96,163.98,13,0,1,4,163.95,163.99,20,121,1,3,0.0,163.98,163.96,48,13,4,1,0.0,NaN,0.57377,6,0.0,-26.0,OK,limite_épuisée
1,2024-09-27 14:00:00.001848092+00:00,T,A,128,163.98,163.96,163.98,13,48,1,4,163.95,163.99,20,121,1,3,0.0,163.98,163.96,48,13,4,1,0.0,NaN,0.57377,6,0.0,-26.0,OK,new_limite


In [15]:
#df__['ts_diff'] = pd.to_datetime(df__['ts_event']).diff().dt.total_seconds()
df__['price_middle'] = (df__['ask_px_00']-df__['bid_px_00'])/2
df__['price_same'] = np.where(df__['side'] == 'A', df__[f'ask_px_0{limite}'],df__[f'bid_px_0{limite}'])
df__['price_opposite'] = np.where(df__['side'] == 'A', df__[f'bid_px_0{limite}'], df__[f'ask_px_0{limite}'])
df__['size_same'] = np.where(df__['side'] == 'A', df__[f'ask_sz_0{limite}'],df__[f'bid_sz_0{limite}'])
df__['size_opposite'] = np.where(df__['side'] == 'A', df__[f'bid_sz_0{limite}'], df__[f'ask_sz_0{limite}'])
df__['nb_ppl_same'] = np.where(df__['side'] == 'A', df__[f'ask_ct_0{limite}'],df__[f'bid_ct_0{limite}'])
df__['nb_ppl_opposite'] = np.where(df__['side'] == 'A', df__[f'bid_ct_0{limite}'], df__[f'ask_ct_0{limite}'])
#df__.drop(columns=[f'bid_px_0{limite}', f'ask_px_0{limite}', f'bid_sz_0{limite}', f'ask_sz_0{limite}', f'bid_ct_0{limite}', f'ask_ct_0{limite}'], axis=1, inplace=True)
df__['diff_price'] = df__['price_middle'].diff()
df__['time_diff'] = df__['ts_event'].diff().dt.total_seconds()



df__['indice'] = range(len(df__))
df__[f'bid_sz_0{limite}_diff'] = df__[f'bid_sz_0{limite}'].diff()
df__[f'ask_sz_0{limite}_diff'] = df__[f'ask_sz_0{limite}'].diff()
condition_T = (
    (df__['action'] == 'T') &
    (
        ((df__['side'] == 'B') & (df__[f'bid_sz_0{limite}_diff'] == -df__['size'])) |
        ((df__['side'] == 'A') & (df__[f'ask_sz_0{limite}_diff'] == -df__['size']))
    )
)

# Condition pour 'A'
condition_A = (
    (df__['action'] == 'A') &
    (
        ((df__['side'] == 'B') & (df__[f'bid_sz_0{limite}_diff'] == df__['size'])) |
        ((df__['side'] == 'A') & (df__[f'ask_sz_0{limite}_diff'] == df__['size']))
    )
)

# Condition pour 'C'
condition_C = (
    (df__['action'] == 'C') &
    (
        ((df__['side'] == 'B') & (df__[f'bid_sz_0{limite}_diff'] == -df__['size'])) |
        ((df__['side'] == 'A') & (df__[f'ask_sz_0{limite}_diff'] == -df__['size']))
    )
)
df__['status'] = np.where(condition_T | condition_A | condition_C, 'OK', 'NOK')
df__.loc[df__['new_limite'] == 'new_limite', 'time_diff'] = np.nan
df__ = df__[df__['time_diff']>0]
df__ = df__[df__['time_diff'] != np.nan]
df_final = df__[df__['status'] != 'NOK']
df__['Mean_price_diff'] = df__['diff_price'].rolling(window=50).mean().shift(1)
df__['imbalance'] = (df__[f'ask_sz_0{limite}']-df__[f'bid_sz_0{limite}'])/(df__[f'ask_sz_0{limite}']+df__[f'bid_sz_0{limite}'])
#df__['imbalance'] = (df__['size_same']-df__['size_opposite'])/(df__['size_same']+df__['size_opposite'])


df__.loc[df__['new_limite'] == 'new_limite', 'time_diff'] = np.nan
df__.head(1)

,ts_event,action,side,size,price,bid_px_00,ask_px_00,bid_sz_00,ask_sz_00,bid_ct_00,ask_ct_00,bid_px_01,ask_px_01,bid_sz_01,ask_sz_01,bid_ct_01,ask_ct_01,time_diff,price_same,price_opposite,size_same,size_opposite,nb_ppl_same,nb_ppl_opposite,diff_price,Mean_price_diff,imbalance,indice,bid_sz_00_diff,ask_sz_00_diff,status,new_limite,price_middle
2,2024-09-27 14:00:00.001912827+00:00,A,A,25,163.98,163.96,163.98,13,73,1,5,163.95,163.99,20,121,1,3,0.000065,163.98,163.96,73,13,5,1,0.0,NaN,0.697674,2,0,25,OK,n,0.01


In [16]:
df_final = df__

In [17]:
# Vérifier s'il y a des NaN dans la colonne 'price'
has_nan_in_price = df__['Mean_price_diff'].isna().any()

if has_nan_in_price:
    print("La colonne 'price' contient des valeurs NaN.")
else:
    print("La colonne 'price' ne contient pas de valeurs NaN.")


La colonne 'price' contient des valeurs NaN.


In [21]:
df_trades = df_final[101:]#[df_final['action'] =='T']
imb = df_trades ['imbalance'].to_numpy()
price = df_trades ['Mean_price_diff'].to_numpy()#/df_trades ['time_diff'].to_numpy()
indices_trie = np.argsort(imb)

# Application du tri aux deux tableaux
imb_trie = imb[indices_trie]
price_trie = price[indices_trie]

group_size = 20000

imb_trie_groups = [imb_trie[i:i + group_size] for i in range(0, len(imb_trie), group_size)]
price_trie_groups = [price_trie[i:i + group_size] for i in range(0, len(price_trie), group_size)]


# Calculer la moyenne de chaque groupe
imb_trie_means = [np.mean(group) for group in imb_trie_groups]
price_trie_means = [np.mean(group) for group in price_trie_groups]
print(price_trie_means)
fig = go.Figure()
fig.add_trace(go.Scatter(x = imb_trie_means, y = price_trie_means, mode ='lines', name ='Prix', showlegend = True))

fig.update_layout(title=f'Bid-ask', xaxis_title='imbalance', yaxis_title='delta_price', showlegend=True)
fig.show()

[3.156999999999679e-05, 2.0010000000003913e-05, 2.009999999999991e-05, 2.0044999999990143e-05, 1.9499999999990594e-05, 2.6054999999998272e-05, 2.907999999999994e-05, 2.5779999999992326e-05, 2.5749999999995693e-05, 2.8150000000001386e-05, 2.9259999999993096e-05, 3.201999999999886e-05, 3.520999999999633e-05, 3.386999999999911e-05, 4.064000000000594e-05, 3.7564999999993316e-05, 3.591500000000087e-05, 4.070499999999632e-05, 3.7379999999994983e-05, 4.232500000001521e-05, 4.006000000000379e-05, 3.838499999999797e-05, 4.826000000000686e-05, 5.085499999999943e-05, 5.7999999999997175e-05, 5.195500000000451e-05, 4.6115000000000835e-05, 2.989417989417013e-05]


In [ ]:
df_trades = df_final[101:]#[df_final['action'] =='T']

# Calculer la moyenne de chaque groupe
imb_trie_means = [np.mean(group) for group in imb_trie_groups]
price_trie_means = [np.mean(group) for group in price_trie_groups]
print(price_trie_means)
fig = go.Figure()
fig.add_trace(go.Scatter(x = imb_trie_means, y = price_trie_means, mode ='lines', name ='Prix', showlegend = True))

fig.update_layout(title=f'Bid-ask', xaxis_title='imbalance', yaxis_title='delta_price', showlegend=True)
fig.show()

In [ ]:
df___.head(4)

In [ ]:
df_.head(4)

In [ ]:
df_['price_same'] = np.where(df_['side'] == 'A', df_[f'ask_px_0{limite}'],df[f'bid_px_0{limite}'])
df_['price_opposite'] = np.where(df_['side'] == 'A', df[f'bid_px_0{limite}'], df_[f'ask_px_0{limite}'])
df_['size_same'] = np.where(df_['side'] == 'A', df_[f'ask_sz_0{limite}'],df[f'bid_sz_0{limite}'])
df_['size_opposite'] = np.where(df_['side'] == 'A', df[f'bid_sz_0{limite}'], df_[f'ask_sz_0{limite}'])
df_['nb_ppl_same'] = np.where(df_['side'] == 'A', df_[f'ask_ct_0{limite}'],df[f'bid_ct_0{limite}'])
df_['nb_ppl_opposite'] = np.where(df_['side'] == 'A', df[f'bid_ct_0{limite}'], df_[f'ask_ct_0{limite}'])
df_.drop(columns=[f'bid_px_0{limite}', f'ask_px_0{limite}', f'bid_sz_0{limite}', f'ask_sz_0{limite}', f'bid_ct_0{limite}', f'ask_ct_0{limite}'], axis=1, inplace=True)
df_['diff_price'] = df_['price'].diff()
df_['Mean_price_diff'] = df_['diff_price'].rolling(window=10).mean().shift(1)
df_['imbalance'] = (df_['size_same']-df_['size_opposite'])/(df_['size_same']+df_['size_opposite'])
df_['ts_event'] = pd.to_datetime(df_['ts_event'])
df_['time_diff'] = df_['ts_event'].diff().dt.total_seconds()
df_['indice'] = range(len(df_))

df_.head(2)

In [ ]:
df_['trii'] = np.where(df_['time_diff'] == 0, df_['indice'] - 1, df_['indice'])
df_.head(20)

In [ ]:
df_trades = df_[df_['action'] =='T']
imb = df_trades ['imbalance'].to_numpy()
price = df_trades ['Mean_price_diff'].to_numpy()#/df_trades ['time_diff'].to_numpy()
indices_trie = np.argsort(imb)

# Application du tri aux deux tableaux
imb_trie = imb[indices_trie]
price_trie = price[indices_trie]

fig = go.Figure()
fig.add_trace(go.Scatter(x = imb_trie, y = price_trie, mode ='lines', name ='Prix', showlegend = True))

fig.update_layout(title=f'Bid-ask', xaxis_title='imbalance', yaxis_title='delta_price', showlegend=True)
fig.show()

In [ ]:
df_trades = df_[df_['action'] =='T']
price = df_trades['price']
bid_ask = 0.02
df_trades = df_trades[np.abs(df_trades['price_same'] - df_trades['price_opposite']) <= bid_ask]
sze_same = df_trades['size_same'].to_numpy()
sze_opposite = df_trades['size_opposite'].to_numpy()
time = df_trades['ts_event']

In [ ]:


fig = go.Figure()
fig.add_trace(go.Scatter(x = time, y = sze_same, mode ='lines', name ='opposite', showlegend = True))
fig.add_trace(go.Scatter(x = time, y = sze_opposite, mode ='lines', name ='same', showlegend = True))
fig.update_layout(title=f'Trades queue size', xaxis_title='time', yaxis_title='queue size', showlegend=True)
fig.show()
fig = go.Figure()
fig.add_trace(go.Scatter(x = time, y = sze_same-sze_opposite, mode ='lines', showlegend = True))
fig.update_layout(title=f'Trades queue size difference with bid-ask smaller than {bid_ask}', xaxis_title='time', yaxis_title='queue size difference', showlegend=True)
fig.show()